<a href="https://colab.research.google.com/github/y-hiroki-radiotech/llm-final-task/blob/main/inference_vllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -U langchain-community langchain-huggingface vllm triton fugashi unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 44.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━

### プロンプトの作成部分

In [1]:
!git clone https://github.com/y-hiroki-radiotech/llm-final-task.git

fatal: destination path 'llm-final-task' already exists and is not an empty directory.


In [2]:
%cd llm-final-task

/content/llm-final-task


In [3]:
from prompt import PromptStock

prompt_stock = PromptStock()

### jsonファイルの読み込み

In [5]:
import pandas as pd

# JSONLファイルを読み込む
file_path = 'elyza-tasks-100-TV_0.jsonl'
data = pd.read_json(file_path, lines=True)

# DataFrameの表示
data

,task_id,input
0,0,野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。
1,1,花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されていま...
2,2,駅について学ぶ上で知っておくべきポイントは？
3,3,以下の説明を読んだユーザーが感じるであろう疑問は？\n\n水の都ベネチアでは、知られざるエピ...
4,4,観光に最適な列車旅を5つ挙げてください。それぞれの提案には、列車の名前、運行区間、特徴を詳し...
...,...,...
95,95,以下の文章について、筆者がどのような意図でこの文章を書いたか、あなたならどのように解釈します...
96,96,若くして看護師を目指し、緩和ケアの道に進む人たちは患者さんとの向き合い方や感情のコントロール...
97,97,漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。\n\n1....
98,98,次の物語の展開を予想してみましょう。\n\nまる、さんかく、しかくの顔をした3きょうだいがい...


### 新しいデータに対して、タスクのラベルを与える。

In [6]:
from transformers import pipeline

model_name = "hiroki-rad/bert-base-classification-ft"
classify_pipe = pipeline(model=model_name, device="cuda:0")

results: list[dict[str, float | str]] = []
for example in data.itertuples():
    # モデルの予測結果を取得
    model_prediction = classify_pipe(example.input)[0]
    # 正解のラベルIDをラベル名に変換
    results.append( model_prediction["label"])

data["label"] = results
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,task_id,input,label
0,0,野球選手が今シーズン活躍するために取り組むべき5つのことを教えてください。,Task_Solution
1,1,花の都で公共ピアノが設置され、訪れた人々が自由にピアノを弾く様子が定点カメラで撮影されていま...,Creative_Generation
2,2,駅について学ぶ上で知っておくべきポイントは？,Knowledge_Explanation
3,3,以下の説明を読んだユーザーが感じるであろう疑問は？\n\n水の都ベネチアでは、知られざるエピ...,Information_Extraction
4,4,観光に最適な列車旅を5つ挙げてください。それぞれの提案には、列車の名前、運行区間、特徴を詳し...,Analytical_Reasoning
...,...,...,...
95,95,以下の文章について、筆者がどのような意図でこの文章を書いたか、あなたならどのように解釈します...,Task_Solution
96,96,若くして看護師を目指し、緩和ケアの道に進む人たちは患者さんとの向き合い方や感情のコントロール...,Task_Solution
97,97,漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。\n\n1....,Information_Extraction
98,98,次の物語の展開を予想してみましょう。\n\nまる、さんかく、しかくの顔をした3きょうだいがい...,Creative_Generation


### モデルのインストール

In [7]:
from langchain_community.llms import VLLM

model_name = "hiroki-rad/llm-jp-llm-jp-3-13b-16-ft"

llm = VLLM(model=model_name,
           quantization="awq")


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

INFO 12-10 21:30:34 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 12-10 21:30:34 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='hiroki-rad/llm-jp-llm-jp-3-13b-16-ft', speculative_config=None, tokenizer='hiroki-rad/llm-jp-llm-jp-3-13b-16-ft', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=hiroki

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

...

model-00004-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 12-10 21:32:50 model_runner.py:1077] Loading model weights took 25.5730 GB
INFO 12-10 21:32:50 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=26.54GiB peak_torch_memory=26.94GiB memory_usage_post_profile=26.55GiB non_torch_memory=0.55GiB kv_cache_size=8.12GiB gpu_memory_utilization=0.90
INFO 12-10 21:32:51 gpu_executor.py:113] # GPU blocks: 665, # CPU blocks: 327
INFO 12-10 21:32:51 gpu_executor.py:117] Maximum concurrency for 4096 tokens per request: 2.60x
INFO 12-10 21:32:53 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-10 21:32:53 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 1

In [23]:
# テンプレート部分
from langchain.prompts import PromptTemplate

template = """
ユーザー: 質問を良く読んで、適切な回答をしてください。
{context}
質問:{input}
回答:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "input"],
    template_format="f-string"
)

In [19]:
# # テンプレート部分
# from langchain.prompts import PromptTemplate

# template = """
# ユーザー: 質問を良く読んで、適切な回答をしてください。

# {context}

# 質問:{input}

# システム:"""

# prompt = PromptTemplate(
#     template=template,
#     input_variables=["context", "input"],
#     template_format="f-string"
# )

In [24]:
vllm_chain = prompt | llm

In [35]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from tqdm import tqdm

# Create a proper chain
chain = (
    RunnablePassthrough()
    | vllm_chain
    | StrOutputParser()
)

# Process each row with fixed position progress bar
outputs = []
total_rows = len(data)
with tqdm(total=total_rows,
          desc="Processing rows",
          position=0,  # 固定位置を指定
          leave=True   # 処理完了後もバーを残す
         ) as pbar:
    for row in data.itertuples():
        prompt_string = prompt_stock.get_prompt(row.label)

        # Prepare the input dictionary
        input_dict = {
            "context": prompt_string,
            "input": row.input
        }

        # Invoke the chain
        output = chain.invoke(input_dict)
        outputs.append(output)

        # Update progress bar
        pbar.update(1)

Processing rows:   0%|          | 0/100 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:  50%|█████     | 1/2 [00:03<00:03,  3.98s/it, est. speed input: 51.51 toks/s, output: 41.20 toks/s]

Processing rows:   1%|          | 1/100 [00:12<20:00, 12.13s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing rows:   2%|▏         | 2/100 [00:19<15:31,  9.51s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing rows:   3%|▎         | 3/100 [00:32<17:31, 10.84s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing rows:   4%|▍         | 4/100 [00:40<15:33,  9.72s/it]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processi

## 出力部分の作成

In [38]:
import json

# Assuming 'data' and 'outputs' are defined from the previous code

# Create a list to store the JSONL objects
jsonl_data = []

# Iterate through the data and outputs
for i in range(len(data)):
    task_id = data.iloc[i]["task_id"] # Access task_id using the index
    output = outputs[i]

    # Create a dictionary for each row
    jsonl_object = {
        "task_id": task_id,
        "output": output
    }
    jsonl_data.append(jsonl_object)

with open("output.jsonl", "w", encoding="utf-8") as outfile:
  for entry in jsonl_data:
    # Convert task_id to a regular Python integer before dumping
    entry["task_id"] = int(entry["task_id"])
    json.dump(entry, outfile, ensure_ascii=False)
    outfile.write('\n')

In [26]:
results

['1. 最適な食事とトレーニングのセットアップ:選手は試合に向けて最高の準備を整えるために、適切な食事とトレーニングが必要です。試合でのパフォーマンスを最大限に引き出すために、体の状態を整えることを重視してください。具体的には、試合の予定を考慮しながら、試合の2-3日前に炭水化物を減らし、試合の直前には1-2時間前に軽い食事を摂ることを提案します。そして、試合後は早期に食事を摂ることを推奨します。さらに、試合に向けて試合の形式に合った日々のトレーニングを行い、試合でのパフォーマンスを最大限に引き出すことが重要です。\n2. 最適な睡眠のタイミング:選手は試合に向けて良い状態を保つために、最適な睡眠が必要です。試合の夜は十分な睡眠をとり、試合前に2時間ほど睡眠時間を増やすことを提案します。試合に向けた体の調整の一環として、試合の数日前に睡眠時間を少し減らすことも考慮してください。最終的に試合の3日前に適切な睡眠スケジュールを設定し、試合に向けて良い睡眠習慣を整えることを目指します。\n3. 最適な試合のセットアップ:選手は試合に向けて最高の準備を整えるために、試合の前には試合で使用する道具の準備、試合の形式を確認し、試合に向けて気持ちを整えることを提案します。試合の前には必要な道具が全て整っているかを確認し、試合の形式を分析して、試合に備えてください。試合前にはリラックスし、試合でのパフォーマンスに集中することが重要です。\n4. 最適な試合運び:選手は試合の鍵となるポイントを見極め、戦略を立てることが重要です。試合の前に相手チームを分析し、試合の展開を予測してください。試合中は自分の役割を果たし、チームの勝利に貢献することを目指してください。試合の状況に応じて適切な判断を行い、チームの勝利のために行動してください。\n5. 最適な試合後の準備:選手は試合後の成功につながる行動が重要です。試合の後は自分自身を評価し、試合での成功と課題を分析してください。試合でのフィードバックを活かし、次の試合に向けて改善点を明確にすることが重要です。また、試合後の身体のケアにも注意し、試合での疲れを癒し、試合の疲れを取り除くための時間を十分に確保してください。\n\n']